# Dyna-Q+ - Planning and Learning

![title](dynaqplus_pic.png)

# Load Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# House Keeping

In [2]:
# Set a Seed
np.random.seed(802)

# Grid size
grid_rows = 6
grid_cols = 9

# Starting maze position
start = [5,3]

# Terminal state
goal = [0,8]

# Obstacles in the maze
walls = [ [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8] ]

# Action set 0=Up 1=Down 2=Left 3=Right
actions = range(4)

# Reward at Each Step
reward = 0

# Discount rate
gamma = 0.95

# Step Size
alpha = 0.1

# Exploration Rate
epsilon = 0.1

# Set number of episodes to run
episodes = 1000

# Number of steps before the environment changes
steps_change = 500

# Number of planning steps
n_planning_steps = 10

# Greedy Action Function

In [3]:
def greedy(state, q_table, epsilon):
    
    # Randomly select action with probability epsilon
    if np.random.uniform() < epsilon:
        g_action = np.random.choice(actions)
        
    # Select the best action for the given state with probability 1-epsilon
    else:
        q_vals = q_table[state[0], state[1], :]
        q_vals_max = np.where(q_vals==q_vals.max())[0]
        g_action = np.random.choice(q_vals_max)
    
    return g_action

# Step Function

In [4]:
def step(state, action, iters, steps_change):
    
    # Define the maze walls based on iteration number
    if iters<steps_change:
        maze_walls = walls
    else:
        maze_walls = walls[:-1]
    
    # Get location
    rownum, colnum = state
    
    # 0=Up
    if action==0:
        rownum = max( rownum - 1, 0 )
    
    # 1=Down
    elif action==1:
        rownum = min( rownum + 1, grid_rows - 1 )
    
    # 2=Left
    elif action==2:
        colnum = max( colnum - 1, 0 )
    
    # 3=Right
    else:
        colnum = min( colnum + 1, grid_cols - 1 )
    
    # Correct for walls
    if [rownum, colnum] in maze_walls:
        rownum, colnum = state
    
    # Specify reward
    if [rownum, colnum] == goal:
        reward = 1
    else:
        reward = 0
    
    return [rownum, colnum], reward

# Q Table

In [5]:
q_table_ql = np.zeros([grid_rows, grid_cols, len(actions)])

# Q-Learning

In [6]:
# Store the steps to reach the goal for each episode
steps_vec_ql = []

# Q-Learning
for i in range(episodes):
    
    # Count steps per episode
    steps_count = 0
    
    # Always start in the same position
    state = start
    
    # Loop until the goal is reached
    while state!=goal:
        
        # Choose action based on epsilon-greedy
        action = greedy(state, q_table_ql, epsilon)
        
        # Find next state given current state and action
        new_state, reward = step(state, action, i, steps_change)
        
        # Update the q-table with Q-Learning
        q_table_ql[state[0],state[1],action] = q_table_ql[state[0],state[1],action] + \
            alpha * ( reward + gamma * max(q_table_ql[new_state[0],new_state[1],]) - \
            q_table_ql[state[0],state[1],action] )
            
        # Set new state to current state
        state = new_state
        
        # Update steps count
        steps_count += 1
    
    # Save steps per episode
    steps_vec_ql.append(steps_count)

# Function to Sample State, Action and Reward, Next State Pairs

In [7]:
def sample_sars_pairs(model):
    
    # Sample random index 
    index_sample = np.random.choice(range(len(model)))
    
    # Find the pairs from the dictionary keys and values
    state_action = list(model.keys())[index_sample]
    newstate_reward = list(model.values())[index_sample]
    
    # Specify values to return
    state = [state_action[0], state_action[1]]
    action = state_action[2]
    new_state = [newstate_reward[0], newstate_reward[1]]
    reward = newstate_reward[2]
    tau = newstate_reward[3]
    
    return state, action, new_state, reward, tau

# Q Table and Evironment Model

In [8]:
q_table_dynaq = np.zeros([grid_rows, grid_cols, len(actions)])
model_dynaq = dict()

# Dyna-Q

In [9]:
# Store the steps to reach the goal for each episode
steps_vec_dynaq = []

# Number of steps taken to map environment
time_steps = 0

# Q-Learning
for i in tqdm(range(episodes)):
    
    # Count steps per episode
    steps_count = 0
    
    # Always start in the same position
    state = start
    
    # Loop until the goal is reached
    while state!=goal:
        
        # Choose action based on epsilon-greedy
        action = greedy(state, q_table_dynaq, epsilon)
        
        # Find next state given current state and action
        new_state, reward = step(state, action, i, steps_change)
        
        # Update the q-table with Q-Learning
        q_table_dynaq[state[0],state[1],action] = q_table_dynaq[state[0],state[1],action] + \
            alpha * ( reward + gamma * max(q_table_dynaq[new_state[0],new_state[1],]) - \
            q_table_dynaq[state[0],state[1],action] )
        
        #####################################################################################
        
        # Create tuples to map environment
        state_action_key = tuple([state[0], state[1], action])
        newstate_reward_val = tuple([new_state[0], new_state[1], reward, 0])

        # Add state, action, reward, new_state to model
        if state_action_key not in model_dynaq.keys():
            model_dynaq[state_action_key] = newstate_reward_val

        #####################################################################################
            
        # Set new state to current state
        state = new_state
        
        # Update steps count
        steps_count += 1
    
    # Save steps per episode
    steps_vec_dynaq.append(steps_count)
    
    # Update q_table with planning
    for j in range(n_planning_steps):
        
        # Sample state_action and reward_nextstate pair
        s_now, action, s_next, reward, tau = sample_sars_pairs(model_dynaq)
        
        # Update the q-table
        q_table_dynaq[s_now[0],s_now[1],action] = q_table_dynaq[s_now[0],s_now[1],action] + \
            alpha * ( reward + gamma * max(q_table_dynaq[s_next[0],s_next[1],]) - \
            q_table_dynaq[s_now[0],s_now[1],action] )

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 357.04it/s]


# Q Table and Evironment Model

In [10]:
q_table_dynaq_plus = np.zeros([grid_rows, grid_cols, len(actions)])
model_dynaq_plus = dict()

# Dyna-Q+

In [11]:
# Store the steps to reach the goal for each episode
steps_vec_dynaq_plus = []

# Number of steps taken to map environment
time_steps = 0

# Small constant -> new reward = reward + kappa * sqrt(tau)
kappa = 0.01

# Q-Learning
for i in tqdm(range(episodes)):
    
    # Count steps per episode
    steps_count = 0
    
    # Always start in the same position
    state = start
    
    # Loop until the goal is reached
    while state!=goal:
        
        # Choose action based on epsilon-greedy
        action = greedy(state, q_table_dynaq_plus, epsilon)
        
        # Find next state given current state and action
        new_state, reward = step(state, action, i, steps_change)
        
        # Update the q-table with Q-Learning
        q_table_dynaq_plus[state[0],state[1],action] = q_table_dynaq_plus[state[0],state[1],action] + \
            alpha * ( reward + gamma * max(q_table_dynaq_plus[new_state[0],new_state[1],]) - \
            q_table_dynaq_plus[state[0],state[1],action] )
        
        #####################################################################################
        
        # Create tuples to map environment
        state_action_key = tuple([state[0], state[1], action])
        newstate_reward_val = tuple([new_state[0], new_state[1], reward, time_steps])

        # Add state, action, reward, new_state to model
        model_dynaq_plus[state_action_key] = newstate_reward_val

        #####################################################################################
            
        # Set new state to current state
        state = new_state
        
        # Update steps count
        steps_count += 1
        
    # Save steps per episode
    steps_vec_dynaq_plus.append(steps_count)
    
    # Update q_table with planning
    for j in range(n_planning_steps):
        
        # Time steps counter
        time_steps += 1
        
        # Sample state_action and reward_nextstate pair
        s_now, action, s_next, reward, tau = sample_sars_pairs(model_dynaq_plus)
        reward = reward + kappa * np.sqrt(time_steps - tau)
        
        # Update the q-table
        q_table_dynaq_plus[s_now[0],s_now[1],action] = q_table_dynaq_plus[s_now[0],s_now[1],action] + \
            alpha * ( reward + gamma * max(q_table_dynaq_plus[s_next[0],s_next[1],]) - \
            q_table_dynaq_plus[s_now[0],s_now[1],action] )

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:16<00:00, 60.60it/s]


# Optimal Policy Function

In [12]:
def optimal_policy(q_table):
    optimal_policy = np.zeros([grid_rows,grid_cols], dtype=str)

    for i in range(grid_rows):
        for j in range(grid_cols):
            direction = np.argmax(q_table[i,j,:])
            if direction==0:
                optimal_policy[i,j]="U"
            elif direction==1:
                optimal_policy[i,j]="D"
            elif direction==2:
                optimal_policy[i,j]="L"
            else:
                optimal_policy[i,j]="R"

    optimal_policy[goal[0],goal[1]] = "G"
    return optimal_policy

In [13]:
print("Q Learning Optimal Policy")
optimal_policy(q_table_ql)

Q Learning Optimal Policy


array([['U', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'G'],
       ['D', 'R', 'R', 'D', 'L', 'R', 'R', 'U', 'L'],
       ['R', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'U'],
       ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'],
       ['U', 'L', 'D', 'D', 'L', 'U', 'U', 'U', 'U'],
       ['R', 'U', 'L', 'L', 'L', 'U', 'U', 'U', 'U']], dtype='<U1')

In [14]:
print("Dyna-Q Optimal Policy")
optimal_policy(q_table_dynaq)

Dyna-Q Optimal Policy


array([['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'G'],
       ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'L'],
       ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'],
       ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'],
       ['U', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L'],
       ['U', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L']], dtype='<U1')

In [15]:
print("Dyna-Q+ Optimal Policy")
optimal_policy(q_table_dynaq_plus)

Dyna-Q+ Optimal Policy


array([['U', 'U', 'L', 'R', 'U', 'U', 'D', 'R', 'G'],
       ['L', 'D', 'U', 'D', 'R', 'R', 'D', 'U', 'U'],
       ['U', 'U', 'D', 'U', 'R', 'D', 'R', 'L', 'U'],
       ['U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U'],
       ['R', 'R', 'L', 'L', 'R', 'R', 'R', 'R', 'U'],
       ['U', 'L', 'L', 'R', 'R', 'U', 'R', 'D', 'L']], dtype='<U1')